In [1]:
from pyrootmemo.models import Waldron
from pyrootmemo.materials import MultipleRoots, Interface, Soil
from pyrootmemo.geometry import SoilProfile, FailureSurface
from pyrootmemo.helpers import units, Parameter

In [2]:
# generate some root data
roots = MultipleRoots(
    species = "test_species",
    diameter = Parameter([1, 2, 1, 3, 4, 5, 6, 3], 'mm'),
    tensile_strength = Parameter([40, 30, 40, 20, 10, 5, 20, 10], 'MPa'),
    elastic_modulus = Parameter([100, 100, 100, 100, 100, 100, 100, 100], 'MPa'),
    length = Parameter([1, 1, 1, 1, 1, 1, 1, 1], 'm'),
    #azimuth_angle = Parameter([0, 0, 0, 0, 0, 0, 0, 0], 'degrees'),
    #elevation_angle = Parameter([-40, -30, -20, -10, 0, 10, 20, 30], 'degrees')
)
# soil--root interface
interface = Interface(
    shear_strength = Parameter(50, "kPa")
)
# generate some soil data
soil = Soil(
    "Sand",
    cohesion = Parameter(2, "kPa"),
    friction_angle = Parameter(30, "degrees")
)
# soil profile object
soil_profile = SoilProfile([soil], depth = Parameter([100], 'm'))
# failure surface object
failure_surface = FailureSurface(
    cross_sectional_area = Parameter(1.0, "m^2"),
    shear_zone_thickness = Parameter(10, "mm"),
    depth = Parameter(1.0, "m")
)

In [3]:
# Create Waldron object
waldron = Waldron(roots, interface, soil_profile, failure_surface)

In [7]:
import numpy as np
u = np.array([1, 2]) * units('mm')
# u = 2 * units('mm')
c = waldron.calc_reinforcement(u, jacobian = True, total = False)
print(c)
# F = waldron.pullout.calc_force(0.1 * units('mm'), jacobian = True)
# print(F)



{'reinforcement': <Quantity([[0.00052869 0.00149536 0.00052869 0.00274716 0.00422953 0.00591094
  0.00777013 0.00274716]
 [0.00119146 0.00336996 0.00119146 0.00619102 0.0095317  0.01332096
  0.01751085 0.00619102]], 'kilopascal')>, 'behaviour_types': array(['Not in tension', 'Anchored, elastic', 'Slipping, elastic',
       'Full pullout', 'Anchored, plastic', 'Slipping, plastic',
       'Slipping, plastic', 'Full pullout'], dtype='<U17'), 'behaviour_fraction': array([[[0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0.]],

       [[0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 

In [5]:
# import matplotlib.pyplot as plt
# cr = waldron.peak_reinforcement()
# print(cr)
# waldron.plot()